In [5]:
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from utils import file_to_coupling_map, directory_to_circuits, build_pass_manager
import random
import time
import csv
import os
import numpy as np




In [6]:
def run_experiment(cm_name, qc_name, layout_trials_list=[1, 10, 100, 1000], heuristic="lookahead", num_pass_trials=5, seed=42, output_csv="experiments/experiment_new.csv"):
    """
    Runs a transpilation experiment using SABRE passes on a set of quantum circuits.

    Parameters:
    - cm_name (str): Name of the coupling map file (without extension) located in 'coupling_maps/' directory.
    - qc_name (str): Name of the directory containing quantum circuits to run the experiment on.
    - layout_trials_list (list): List of layout (and swap) trials to run for each circuit. Default is [1, 10, 100, 1000].
    - heuristic (str): Heuristic to use in SabreSwap. Default is "lookahead".
    - num_pass_trials (int): Number of different seeds for running the whole pass. Default is 1.
    - seed (int): Random seed for reproducibility. Default is 42.
    - output_csv (str): Filename for the CSV file where results will be saved. Default is "experiment_results.csv".
    
    Returns:
    - None (Results are saved to a CSV file)
    """
    
    # Set random seed for reproducibility
    random.seed(seed)

    # Load the coupling map and the quantum circuits
    cm = CouplingMap(file_to_coupling_map(f'coupling_maps/{cm_name}.txt'))
    qc_list = directory_to_circuits(f'{qc_name}')

    # Prepare the CSV file
    with open(output_csv, mode='w', newline='') as csvfile:
        fieldnames = ["num_qubits", "layout_trials", "heuristic", "avg_runtime", "avg_swap_count", "avg_depth", "num_pass_trials"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        # Run trials for each circuit
        for qc in qc_list:
            num_qubits = qc.num_qubits
            print(f"Running experiment for {num_qubits} qubits")

            for layout_trials in layout_trials_list:
                seed_list = random.sample(range(0, 100000), num_pass_trials)

                total_runtime = 0
                total_swap_count = 0
                total_depth = 0

                for trial in range(num_pass_trials):
                    # Create the pass manager with the specified layout trials
                    rp = SabreSwap(coupling_map=cm, heuristic=heuristic, seed=seed_list[trial])
                    lp = SabreLayout(coupling_map=cm, layout_trials=trial, swap_trials=trial, seed=seed_list[trial], max_iterations=4)
                    pm = build_pass_manager(cm, lp, rp)

                    # Run the transpilation pass
                    time_start = time.time()
                    qc_tr = pm.run(qc)
                    time_end = time.time()

                    # Collect the metrics for this trial
                    runtime = time_end - time_start
                    swap_count = qc_tr.count_ops().get('swap', 0)
                    depth = qc_tr.depth(lambda x: x.operation.num_qubits == 2)  # 2-qubit gates

                    total_runtime += runtime
                    total_swap_count += swap_count
                    total_depth += depth

                    print(f"Layout trials: {layout_trials}, Seed: {seed_list[trial]} - Trial {trial+1}/{num_pass_trials} - Runtime: {runtime:.10f} sec, Swap count: {swap_count}, Depth: {depth}")

                # Calculate averages
                avg_runtime = total_runtime / num_pass_trials
                avg_swap_count = total_swap_count / num_pass_trials
                avg_depth = total_depth / num_pass_trials

                # Store averaged results in CSV
                writer.writerow({
                    "num_qubits": num_qubits,
                    "layout_trials": layout_trials,
                    "heuristic": heuristic,
                    "avg_runtime": avg_runtime,
                    "avg_swap_count": avg_swap_count,
                    "avg_depth": avg_depth,
                    "num_pass_trials": num_pass_trials
                })
                csvfile.flush()

    print(f"Results saved to {output_csv}")

In [7]:
import os

# Set the RUST_BACKTRACE environment variable
os.environ['RUST_BACKTRACE'] = '1'

In [8]:
# Example usage:

heuristic = "basic"

# Generate log scale values
layout_trials_list = [1, 2]

run_experiment(cm_name='heavy_hex_127', qc_name='qft', heuristic=heuristic, layout_trials_list=layout_trials_list, num_pass_trials=1, output_csv=f"experiments/qft_new.csv")



Running experiment for 10 qubits


thread '<unnamed>' panicked at crates/accelerate/src/sabre/route.rs:740:14:
called `Option::unwrap()` on a `None` value
stack backtrace:
   0: rust_begin_unwind
             at /rustc/8bfcae730a5db2438bbda72796175bba21427be1/library/std/src/panicking.rs:665:5
   1: core::panicking::panic_fmt
             at /rustc/8bfcae730a5db2438bbda72796175bba21427be1/library/core/src/panicking.rs:74:14
   2: core::panicking::panic
             at /rustc/8bfcae730a5db2438bbda72796175bba21427be1/library/core/src/panicking.rs:148:5
   3: core::option::unwrap_failed
             at /rustc/8bfcae730a5db2438bbda72796175bba21427be1/library/core/src/option.rs:2020:5
   4: core::option::Option<T>::unwrap
             at /rustc/8bfcae730a5db2438bbda72796175bba21427be1/library/core/src/option.rs:970:21
   5: qiskit_accelerate::sabre::route::swap_map
             at /Users/henryzou/Documents/GitHub/qiskit/crates/accelerate/src/sabre/route.rs:725:9
   6: qiskit_accelerate::sabre::layout::layout_trial
          

PanicException: called `Option::unwrap()` on a `None` value